In [ ]:
import numpy as np, matplotlib.pyplot as plt, cv2, tqdm
%matplotlib inline

In [ ]:
from utils import saveVideo, show, drawShape

In [ ]:
import laneFindingPipeline

In [ ]:
# Load video
fpath = 'project_video.mp4'
import skvideo.io
reader = skvideo.io.FFmpegReader(fpath)
frames = []
for (i, f) in enumerate(tqdm.tqdm_notebook(
    reader.nextFrame(), 
    total=reader.inputframenum,
)):
    frames.append(f)

In [ ]:
laneFinder = laneFindingPipeline.LaneFinder()

In [ ]:
y = 720

bases = [[], []]
yall = [[], []]
for frame in tqdm.tqdm_notebook(frames[:256], unit='frame'):
    warped = laneFinder.preprocess(frame)
    left, right = laneFinder.initialDiscovery(warped)
    left, right = [
        laneFindingPipeline.LaneMarking(points)
        for points in (left, right)
    ]
    
    # Extract the lane bases x(y=720).
    bases[0].append(left(720))
    bases[1].append(right(720))
    
    yall[0].append(left.y)
    yall[1].append(right.y)

In [ ]:
hist2mode = lambda binCounts, binEdges, _: binEdges[np.argmax(binCounts):][:2].mean()

In [ ]:
fig, ax = plt.subplots()
nbins = 128

alpha = .25
    
dcolor = 'red'
mode = hist2mode(*ax.hist(
    np.array(bases[1]) - np.array(bases[0]), 
    bins=nbins, label='$\Delta$', alpha=alpha, color=dcolor
))
ax.axvline(mode, label='$\Delta$ mode (%.4g)' % mode, color=dcolor)

for baselist, label in zip(bases, ['left', 'right']):
    ax.hist(baselist, bins=nbins, label=label, alpha=alpha)

ax.legend()
ax.set_xlabel('x(y=%d) location [pixels]' % y)
ax.set_ylabel('count');
fig.savefig('doc/laneBaseDifferenceHistogram.png')

In [ ]:
mode

In [ ]:
xm_per_pix = 3.7 / mode

In [ ]:
laneLengths = [[], []]    
for i in tqdm.tqdm_notebook(range(len(yall[0]))):
    # Extract the lane lengths (spans).
    for ll, laney in zip(laneLengths, [yall[0][i], yall[1][i]]):
        ypix = set(laney)
        ll.append(max(laney) - min(laney))

In [ ]:
fig, ax = plt.subplots()
nbins = 32
alpha = .1
    
meanLengths = sum([np.array(ll) for ll in laneLengths]) / len(laneLengths)
for ll, label in zip(
    [laneLengths[0], laneLengths[1], meanLengths], 
    ['left', 'right', 'mean']
):
    ll = np.array(ll)
    ll = ll[ll < 715]
    a, b, c = ax.hist(ll, bins=nbins, label=label, alpha=alpha)
    color = c[0].get_facecolor()[:-1]
    mode = hist2mode(a,b,c)
    ax.axvline(mode, color=color, label='mode of %ss (%.4g)' % (label, mode))
    
ax.legend(fontsize=16)
ax.set_xlabel('span of pixel y coordinates $y_{max}-y_{min}$ [pixels]')
ax.set_ylabel('count');
fig.savefig('doc/laneHeightHistogram.png')

In [ ]:
mode

In [ ]:
ym_per_pix = 30 / mode

In [ ]:
frame = frames[400]
warped = laneFinder.preprocess(frame)
show(warped);

In [ ]:
left, right = laneFinder(frame)

In [ ]:
left.radius, right.radius

In [ ]:
left, right = laneFinder.initialDiscovery(warped)
left, right = [
    laneFindingPipeline.LaneMarking(points)
    for points in (left, right)
]

How many feet wide does this estimate the lane to be?

In [ ]:
(right.radius - left.radius) * 3.2808

Well, it's only about 4x too large -- OOM correct.

In [ ]:
centerline = np.mean([left(720), right(720)])
metersRightOfCenter = (1280/2 - centerline) * left.xm_per_pix

How many feet right of center are we?

In [ ]:
metersRightOfCenter * 3.2808